In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import joblib
import lightgbm as lgb

In [2]:
sns.set_theme()

In [3]:
model = joblib.load('lgbm_model.joblib')

In [4]:
def aggregate_teams(df, min_players=23):
    # Choose columns to keep.
    columns = (['year', 'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'height_cm', 
        'weight_kg', 'nationality_name', 'nation_team_id'] + 
        [f for f in df.columns if f.startswith(('attacking_', 'skill_', 'movement_', 'power_', 
            'mentality_', 'defending_', 'goalkeeping_'))])
    columns.remove('mentality_composure')
    columns.remove('goalkeeping_speed')
    df = df[columns]
    # Teams with defined national team
    df_nat_team = df.dropna(subset=['nation_team_id'])
    df_nat_team = df_nat_team.drop(columns=['nation_team_id'])
    df = df.drop(columns=['nation_team_id'])
    no_nat_names = set(df['nationality_name'].unique()).difference(set(df_nat_team['nationality_name'].unique()))
    # Teams without a defined national team but with at least min_players players
    df_no_nat_team = df[df['nationality_name'].isin(no_nat_names)]\
        .groupby(['year', 'nationality_name'])\
        .filter(lambda g: g.count()['overall'] >= min_players)\
        .sort_values(['overall'], ascending=False)\
        .groupby(['year', 'nationality_name'])\
        .apply(lambda g: g[:min_players])\
        .reset_index(drop=True)
    df = pd.concat((df_nat_team, df_no_nat_team), ignore_index=True)
    # Set nan for value and wage to 0
    df = df.fillna(0)
    # For each feature, aggregate mean, min, and max per team
    df_agg = df.groupby(['year', 'nationality_name']).agg(['mean', 'min', 'max'])
    df_agg.columns = df_agg.columns.map('_'.join)
    keep_columns = [f for f in df_agg.columns if not (f.startswith('goalkeeping_') and f.endswith(('mean', 'min')))]
    df_agg = df_agg[keep_columns].reset_index()
    df_agg = df_agg.rename(columns={'nationality_name': 'team'})
    df_agg['team'] = df_agg['team'].astype('string')
    return df_agg

def merge_wc22_teams(res, teams):
    match_list = []
    val_cols = [c for c in teams.columns if c not in ['year', 'team']]
    col_names = ['team1_' + c for c in val_cols] + ['team2_' + c for c in val_cols]
    for row in res.itertuples(index=False):
        team1 = row.team1
        team2 = row.team2
        col_vals = np.hstack((
            teams[teams['team'] == team1].sort_values('year').iloc[-1][val_cols].values,
            teams[teams['team'] == team2].sort_values('year').iloc[-1][val_cols].values
        ))
        d = {col_name: col_val for (col_name, col_val) in zip(col_names, col_vals)}
        d['team1'] = team1
        d['team2'] = team2
        match_list.append(d)
    return pd.DataFrame(match_list)

def transform_wc22(df):
    df = df[df['Team1_Name'] != '-']
    team1 = np.hstack((df['Team1_Name'].values, df['Team2_Name'].values))
    team2 = np.hstack((df['Team2_Name'].values, df['Team1_Name'].values))
    df = pd.DataFrame({'team1': team1, 'team2': team2})
    df['team1'] = df['team1'].astype('string')
    df['team2'] = df['team2'].astype('string')
    df = df.replace({'IR Iran': 'Iran', 'South Korea': 'Korea Republic', 'USA': 'United States'})
    return df

def predict_wc22(model, df, feature_cols):
    n_matches = len(df) // 2
    y_hat_wc22 = model.predict(df[feature_cols].values)
    y_hat_wc22_proba = model.predict_proba(df[feature_cols].values)
    y_hat_wc22_proba_max = np.max(y_hat_wc22_proba, axis=1)
    df['team1_score_pred'] = y_hat_wc22
    df['team1_score_pred_proba'] = y_hat_wc22_proba_max
    df['team2_score_pred'] = df['team1_score_pred'].shift(-n_matches)
    df['team2_score_pred_proba'] = df['team1_score_pred_proba'].shift(-n_matches)
    df = df.iloc[0:n_matches][['team1', 'team2', 'team1_score_pred', 'team2_score_pred', 'team1_score_pred_proba', 'team2_score_pred_proba']]
    df['team2_score_pred'] = df['team2_score_pred'].astype(int)
    df['pred_proba'] = df['team1_score_pred_proba'] * df['team2_score_pred_proba']
    df = df[['team1', 'team2', 'team1_score_pred', 'team2_score_pred', 'pred_proba']]
    return df

In [5]:
x_cols = ['overall', 'potential', 'skill_moves', 'attacking_finishing', 
          'skill_long_passing', 'movement_sprint_speed', 'movement_agility', 
          'value_eur', 'wage_eur', 'attacking_finishing', 'power_stamina']
x_cols = [s + a for s in x_cols  for a in ['_min', '_mean', '_max']]
x_cols = x_cols + ['goalkeeping_positioning_max', 'goalkeeping_reflexes_max']
x_cols = [t + s for t in ['team1_', 'team2_'] for s in x_cols]

In [6]:
from itertools import combinations
df_wc_teams = pd.read_csv('Teams.csv')
df_wc_teams = df_wc_teams.replace({'USA': 'United States', 'IR Iran': 'Iran', 'South Korea': 'Korea Republic'})
df_matches = pd.read_csv('Matches.csv')
res_wc22 = transform_wc22(df_matches)
df_teams = pd.read_csv('players_all_years.csv')
df_teams = aggregate_teams(df_teams, min_players=1)
df_wc22 = merge_wc22_teams(res_wc22, df_teams)
df_all_matches = pd.DataFrame({'Team1_Name': x[0], 'Team2_Name': x[1]} for x in combinations(df_wc_teams['Team_Name'].values, 2))
res_all = transform_wc22(df_all_matches)
df_wc22_all = merge_wc22_teams(res_all, df_teams)

C:\Users\pejacobs\AppData\Local\Temp\ipykernel_22044\3812602579.py:6: DtypeWarning: Columns (26,109) have mixed types. Specify dtype option on import or set low_memory=False.
  df_teams = pd.read_csv('players_all_years.csv')


In [19]:
df_wc22_all

,team1,team2,team1_score_pred,team2_score_pred,pred_proba,Group
0,Qatar,Ecuador,1,2,0.161747,A
1,Qatar,Senegal,0,3,0.199172,A
2,Qatar,Netherlands,0,3,0.150047,A
3,Qatar,England,0,5,0.389004,-
4,Qatar,Iran,0,2,0.141219,-
...,...,...,...,...,...,...
491,Portugal,Uruguay,3,0,0.227951,H
492,Portugal,Korea Republic,3,0,0.208465,H
493,Ghana,Uruguay,1,1,0.133734,H
494,Ghana,Korea Republic,1,0,0.088214,H


In [14]:
df_teams[df_teams['year'] == 2022]

,year,team,overall_mean,overall_min,overall_max,potential_mean,potential_min,potential_max,value_eur_mean,value_eur_min,...,defending_standing_tackle_min,defending_standing_tackle_max,defending_sliding_tackle_mean,defending_sliding_tackle_min,defending_sliding_tackle_max,goalkeeping_diving_max,goalkeeping_handling_max,goalkeeping_kicking_max,goalkeeping_positioning_max,goalkeeping_reflexes_max
1087,2022,Afghanistan,64.000000,64,64,69.000000,69,69,8.750000e+05,875000.0,...,33,33,35.000000,35,35,13,6,10,10,7
1088,2022,Albania,80.000000,80,80,83.000000,83,83,2.100000e+07,21000000.0,...,18,18,12.000000,12,12,82,77,74,77,84
1089,2022,Algeria,86.000000,86,86,86.000000,86,86,6.550000e+07,65500000.0,...,31,31,22.000000,22,22,15,9,13,11,6
1090,2022,Andorra,64.000000,64,64,64.000000,64,64,4.000000e+05,400000.0,...,62,62,64.000000,64,64,9,7,12,8,6
1091,2022,Angola,78.000000,78,78,78.000000,78,78,1.250000e+07,12500000.0,...,77,77,76.000000,76,76,10,10,7,10,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226,2022,Uzbekistan,73.000000,73,73,73.000000,73,73,3.200000e+06,3200000.0,...,38,38,34.000000,34,34,15,5,5,7,5
1227,2022,Vietnam,64.000000,64,64,66.000000,66,66,5.250000e+05,525000.0,...,10,10,10.000000,10,10,65,61,58,65,65
1228,2022,Wales,72.043478,63,82,76.652174,64,82,6.328261e+06,425000.0,...,16,81,50.652174,13,80,76,74,77,74,75
1229,2022,Zambia,77.000000,77,77,84.000000,84,84,2.150000e+07,21500000.0,...,24,24,16.000000,16,16,8,6,9,5,11


In [8]:
df_wc22_all = predict_wc22(model, df_wc22_all, x_cols)

In [10]:
def get_group(row, df_wc_teams):
    g1 = df_wc_teams.loc[df_wc_teams['Team_Name'] == row['team1'], 'GroupName'].values[0]
    g2 = df_wc_teams.loc[df_wc_teams['Team_Name'] == row['team2'], 'GroupName'].values[0]
    if g1 == g2:
        return g1
    else:
        return '-'

In [11]:
df_wc22_all['Group'] = df_wc22_all.apply(lambda r: get_group(r, df_wc_teams), axis=1)

In [12]:
df_wc22_all

,team1,team2,team1_score_pred,team2_score_pred,pred_proba,Group
0,Qatar,Ecuador,1,2,0.161747,A
1,Qatar,Senegal,0,3,0.199172,A
2,Qatar,Netherlands,0,3,0.150047,A
3,Qatar,England,0,5,0.389004,-
4,Qatar,Iran,0,2,0.141219,-
...,...,...,...,...,...,...
491,Portugal,Uruguay,3,0,0.227951,H
492,Portugal,Korea Republic,3,0,0.208465,H
493,Ghana,Uruguay,1,1,0.133734,H
494,Ghana,Korea Republic,1,0,0.088214,H


In [ ]:
teams = (('Senegal', 'United States'), ('Argentina', 'Denmark'))


In [169]:
eighth = [('Senegal', 'United States'), ('Argentina', 'Denmark'), ('Spain', 'Croatia'), 
    ('Switzerland', 'Uruguay'), ('England', 'Netherlands'), ('France', 'Poland'), 
    ('Belgium', 'Germany'), ('Portugal', 'Brazil')]
quarter = [('United States', 'Argentina'), ('Spain', 'Switzerland'), ('Netherlands', 'France'), ('Germany', 'Portugal')]
semi = [('Argentina', 'Spain'), ('France', 'Portugal')]
final = [('Argentina', 'Portugal'), ('Spain', 'France')]
for (t1, t2) in final:
    idx = ((df_wc22_all['team1'] == t1) & (df_wc22_all['team2'] == t2)) | ((df_wc22_all['team1'] == t2) & (df_wc22_all['team2'] == t1))
    display(df_wc22_all[idx])

,team1,team2,team1_score_pred,team2_score_pred,pred_proba,Group
239,Argentina,Portugal,0,0,0.099535,-


,team1,team2,team1_score_pred,team2_score_pred,pred_proba,Group
309,France,Spain,1,1,0.19055,-


In [164]:
df_wc22_all[df_wc22_all['team1'] == 'Netherlands']

,team1,team2,team1_score_pred,team2_score_pred,pred_proba,Group
90,Netherlands,England,1,1,0.132856,-
91,Netherlands,Iran,3,1,0.156466,-
92,Netherlands,United States,4,0,0.095921,-
93,Netherlands,Wales,3,1,0.241871,-
94,Netherlands,Argentina,0,2,0.108815,-
95,Netherlands,Saudi Arabia,4,0,0.163439,-
96,Netherlands,Mexico,3,2,0.094017,-
97,Netherlands,Poland,4,2,0.143401,-
98,Netherlands,France,1,1,0.112052,-
99,Netherlands,Australia,4,0,0.158154,-
